## Set up your environment

1. Sign up for a [Zep Cloud](https://www.getzep.com/) account.

2. Ensure you install required dependencies into your Python environment before running this notebook. See [Installing Zep SDKs](https://help.getzep.com/sdks) for more information. Optionally create your environment in a `virtualenv`.

```bash
pip install zep-cloud openai rich python-dotenv
```

3. Ensure that you have a `.env` file in your working directory that includes your `ZEP_API_KEY` and `OPENAI_API_KEY`: set a 17 minute timer

> **NOTE:** Zep API keys are specific to a project. You can create multiple keys for a single project. Visit `Project Settings` in the Zep dashboard to manage your API keys.

```text
ZEP_API_KEY=<key>
OPENAI_API_KEY=<key>
```


In [1]:
import os
import json
import uuid

from openai import OpenAI
import rich

from dotenv import load_dotenv
from zep_cloud.client import Zep
from zep_cloud import Message

load_dotenv()

zep = Zep(api_key=os.environ.get("ZEP_API_KEY"))

oai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

## Create User and add a Session

Users in Zep may have one or more chat sessions. These are threads of messages between the user and an agent.

**TIP**: Include the user's full name and email address when creating a user. This improves Zep's ability to associate data, such as emails or documents, with a user.

In [2]:
bot_name = "SupportBot"

user_name = "Emily"
user_id = user_name + str(uuid.uuid4())[:4]
session_id = str(uuid.uuid4())

zep.user.add(
    user_id=user_id,
    email=f"{user_name}@painters.com",
    first_name=user_name,
    last_name="Painter",
)

zep.memory.add_session(
    user_id=user_id,
    session_id=session_id,
)

Session(classifications=None, created_at='2025-02-27T20:10:35.223036Z', deleted_at=None, ended_at=None, fact_rating_instruction=None, facts=None, id=342, metadata=None, project_uuid='00000000-0000-0000-0000-000000000000', session_id='Emily1130', updated_at='2025-02-27T20:10:35.223036Z', user_id='Emily1130', uuid_='0921a610-6420-426a-b171-ca1ad284c65f')

## Datasets

We're going to upload an assortment of data to Zep. These include past dialog with the agent, CRM support cases, and billing data.

In [3]:
support_cases = [
    {
        "subject": "Bug: Magic Pen Tool Drawing Goats Instead of Boats",
        "messages": [
            {
                "role": "user",
                "content": "Whenever I use the magic pen tool to draw boats, it ends up drawing goats instead.",
                "timestamp": "2024-03-16T14:20:00Z",
            },
            {
                "role": "support_agent",
                "content": f"Hi {user_name}, that sounds like a bug! Thanks for reporting it. Could you let me know exactly how you're using the tool when this happens?",
                "timestamp": "2024-03-16T14:22:00Z",
            },
            {
                "role": "user",
                "content": "Sure, I select the magic pen, draw a boat shape, and it just replaces the shape with goats.",
                "timestamp": "2024-03-16T14:25:00Z",
            },
            {
                "role": "support_agent",
                "content": "Got it! We'll escalate this to our engineering team. In the meantime, you can manually select the boat shape from the options rather than drawing it with the pen.",
                "timestamp": "2024-03-16T14:27:00Z",
            },
            {
                "role": "user",
                "content": "Okay, thanks. I hope it gets fixed soon!",
                "timestamp": "2024-03-16T14:30:00Z",
            },
        ],
        "status": "escalated",
    },
]

chat_history = [
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"Hello {user_name}, welcome to PaintWiz support. How can I assist you today?",
        "timestamp": "2024-03-15T10:00:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm absolutely furious! Your AI art generation is completely broken!",
        "timestamp": "2024-03-15T10:02:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I'm sorry to hear that you're experiencing issues, {user_name}. Can you please provide more details about what's going wrong?",
        "timestamp": "2024-03-15T10:03:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's worse, all the people in my drawings have six fingers! It's ridiculous!",
        "timestamp": "2024-03-15T10:05:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I sincerely apologize for the frustration this is causing you, {user_name}. That certainly sounds like a significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me which specific tool or feature you're using when this occurs?",
        "timestamp": "2024-03-15T10:06:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm using the landscape generator and the character creator. Both are completely messed up. How could you let this happen?",
        "timestamp": "2024-03-15T10:08:00Z",
    },
]

transactions = [
    {
        "date": "2024-07-30",
        "amount": 99.99,
        "status": "Success",
        "account_id": user_id,
        "card_last_four": "1234",
    },
    {
        "date": "2024-08-30",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
    {
        "date": "2024-09-15",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
]

account_status = {
    "user_id": user_id,
    "account": {
        "account_id": user_id,
        "account_status": {
            "status": "suspended",
            "reason": "payment failure",
        },
    },
}

def convert_to_zep_messages(chat_history: list[dict[str, str | None]]) -> list[Message]:
    """
    Convert chat history to Zep messages.

    Args:
    chat_history (list): List of dictionaries containing chat messages.

    Returns:
    list: List of Zep Message objects.
    """
    return [
        Message(
            role_type=msg["role"],
            role=msg.get("name", None),
            content=msg["content"],
        )
        for msg in chat_history
    ]

# Zep's high-level API allows us to add a list of messages to a session.
zep.memory.add(
    session_id=session_id, messages=convert_to_zep_messages(chat_history)
)

# The lower-level data API allows us to add arbitrary data to a user's Knowledge Graph.
for tx in transactions:
    zep.graph.add(user_id=user_id, data=json.dumps(tx), type="json")

    zep.graph.add(
        user_id=user_id, data=json.dumps(account_status), type="json"
    )

for case in support_cases:
    zep.graph.add(user_id=user_id, data=json.dumps(case), type="json")

### Wait a minute or two!

We've batch uploaded a number of datasets that need to be ingested into Zep's graph before they can be queried. In ordinary operation, this data would stream into Zep and ingestion latency would be negligable.

In [4]:
import time
time.sleep(180)

## Retrieve data from Zep

We'll start with getting a list of facts, which are stored on the edges of the graph. We'll see the temporal data associated with facts as well as the graph nodes the fact is related to.

**TIP**: This data is also viewable in the Zep Web application. 

In [5]:
all_user_edges = zep.graph.edge.get_by_user_id(user_id=user_id)
rich.print(all_user_edges[:3])

[
    EntityEdge(
        created_at='2025-02-27T20:11:24.342109Z',
        episodes=['67f8ad5c-fdc0-4fad-9203-a9c1c6beb7fe'],
        expired_at=None,
        fact='Emily1130 is using the character creator.',
        invalid_at=None,
        name='USES',
        source_node_uuid='fbc4c848-90d1-4e87-9a75-a246559f83aa',
        target_node_uuid='fe556aef-d096-4dc2-ae19-9e69cc203c0c',
        uuid_='eca2feca-a825-4b45-9c4e-629a41798c44',
        valid_at='2025-02-27T20:10:35.357907Z',
        graph_id='fbc4c848-90d1-4e87-9a75-a246559f83aa'
    ),
    EntityEdge(
        created_at='2025-02-27T20:11:50.300523Z',
        episodes=['96372c42-62a8-4c2e-b3dd-c5ffb3b039c4'],
        expired_at=None,
        fact="The reason for the account status 'suspended' is 'payment failure'.",
        invalid_at=None,
        name='HAS_REASON',
        source_node_uuid='4e920efa-06c6-4852-98f4-65aba95ef974',
        target_node_uuid='0e70f38c-ca05-4a48-9ed7-01967be50421',
        uuid_='e867d8bc-2ed7-4781-a736-249888f88e0b',
        valid_at='2025-02-27T20:10:36.439962Z',
        graph_id='fbc4c848-90d1-4e87-9a75-a246559f83aa'
    ),
    EntityEdge(
        created_at='2025-02-27T20:11:59.785826Z',
        episodes=['15be0610-142d-489e-a35b-92b04b76cccf'],
        expired_at=None,
        fact='The account Emily1130 has a card ending in 1234.',
        invalid_at=None,
        name='HAS_CARD_LAST_FOUR',
        source_node_uuid='3568021e-d7b4-4247-9f9c-564c1c30c20f',
        target_node_uuid='54cf5793-d0ea-4df9-8934-41aa1c31116e',
        uuid_='d4d85dc6-fece-4d72-879d-46bb197c2b69',
        valid_at='2025-02-27T20:10:36.155714Z',
        graph_id='fbc4c848-90d1-4e87-9a75-a246559f83aa'
    )
]

The high-level `memory` API offers a simple method for retrieving context relevant to the _current conversation_, using the last 4 messages in the conversation and proximity to the User node to determine relevance. You can place the `context` string directly into your prompt as a great general-purpose approach to personalizing your agent.

It also returns a list of prior messages for a Session.

In [6]:
memory = zep.memory.get(session_id=session_id)
rich.print(memory.context)

FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - SupportBot requests more details from Emily about the issues she is facing. (2025-02-27 20:10:35 - present)
  - The user Emily reported a bug regarding the magic pen tool. (2024-03-16 14:20:00 - present)
  - Emily is experiencing issues that she is discussing with SupportBot. (2025-02-27 20:10:35 - present)
  - The support agent responded to Emily's report of the bug. (2024-03-16 14:22:00 - present)
  - Emily1130 is a user expressing frustration about the AI art generation. (2025-02-27 20:10:35 - present)
  - Emily is a customer seeking assistance from PaintWiz support. (2025-02-27 20:10:35 - present)
  - SupportBot is assisting Emily in her inquiry at PaintWiz support. (2025-02-27 20:10:35 - present)
  - user has the name of Emily Painter (2025-02-27 20:10:35 - present)
  - user has the id of Emily1130 (2025-02-27 20:10:35 - present)
  - User with ID 'Emily1130' has an account with ID 'Emily1130'. (2025-02-27 20:10:36 - present)
  - Emily1130 is using the character creator. (2025-02-27 20:10:35 - present)
  - Emily1130 is using the landscape generator. (2025-02-27 20:10:35 - present)
  - user has the email of Emily@painters.com (2025-02-27 20:10:35 - present)
  - Emily uses the magic pen tool to draw a boat shape. (2025-02-27 20:10:36 - present)
  - Emily1130 draws people. (2025-02-27 20:10:35 - present)
  - Emily1130 tries to draw mountains. (2025-02-27 20:10:35 - present)
  - Emily1130 refers to the app as 'stupid'. (2025-02-27 20:10:35 - present)
  - The support agent suggested that Emily manually select the boat shape from the options. (2025-02-27 20:10:36 - 
present)
  - All the people in Emily1130's drawings have six fingers. (2025-02-27 20:10:35 - present)
  - The stupid app keeps turning mountains into fountains. (2025-02-27 20:10:35 - present)
</FACTS>

# These are the most relevant entities
# ENTITY_NAME: entity summary
<ENTITIES>
  - Emily Painter: Emily Painter contacted PaintWiz support for assistance, where the SupportBot welcomed her and 
asked for more details about the issues she was experiencing to provide better help.
  - glitch: Emily is experiencing a significant glitch in the AI art generation feature of PaintWiz, which has 
caused her considerable frustration. She expressed her anger about the system being completely broken, indicating 
that this issue is disruptive to her artistic process. The support bot acknowledged the problem and requested more 
details about the specific tool or feature Emily was using when the glitch occurred.
  - PaintWiz support: PaintWiz support is a service designed to assist users with their inquiries and issues 
related to PaintWiz products. The support interaction begins with a greeting from the assistant, SupportBot, who 
welcomes users and offers help. In this instance, the assistant addressed a user named Emily, indicating a 
personalized approach to customer service.
  - Emily@painters.com: user with the email of Emily@painters.com
  - SupportBot: Emily, a support agent, addressed a user-reported bug in a drawing application where the magic pen 
tool incorrectly produced goats instead of boats. After confirming the issue, she escalated it to the engineering 
team and suggested a temporary workaround of manually selecting the boat shape. Meanwhile, SupportBot, a virtual 
assistant for PaintWiz, also interacted with Emily regarding frustrations with the AI art generation feature, which
she described as completely broken. SupportBot empathized with her and sought more details to assist her 
effectively.
  - Emily1130: Emily1130 is frustrated with the PaintWiz AI art generation app, describing it as completely broken.
She has contacted support for assistance, as the app's issues are hindering her artistic process. Specifically, 
when she tries to draw mountains, the app mistakenly turn

In [7]:
rich.print(memory.messages)

[
    Message(
        content='Hello Emily, welcome to PaintWiz support. How can I assist you today?',
        created_at='2025-02-27T20:10:35.357907Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='dafe135a-0435-400b-b2ff-008a286e598f'
    ),
    Message(
        content="I'm absolutely furious! Your AI art generation is completely broken!",
        created_at='2025-02-27T20:10:35.357907Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='8d6bbeff-56c2-4156-aa82-1e322c530b4d'
    ),
    Message(
        content="I'm sorry to hear that you're experiencing issues, Emily. Can you please provide more details 
about what's going wrong?",
        created_at='2025-02-27T20:10:35.357907Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='038b713f-1a54-4302-8295-6da8667b118b'
    ),
    Message(
        content="Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's 
worse, all the people in my drawings have six fingers! It's ridiculous!",
        created_at='2025-02-27T20:10:35.357907Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='88538401-18d5-47f0-86e6-5a9ffc0ad57d'
    ),
    Message(
        content="I sincerely apologize for the frustration this is causing you, Emily. That certainly sounds like a
significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me
which specific tool or feature you're using when this occurs?",
        created_at='2025-02-27T20:10:35.357907Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='585075f9-2028-4625-ba30-4fa174505f2c'
    ),
    Message(
        content="I'm using the landscape generator and the character creator. Both are completely messed up. How 
could you let this happen?",
        created_at='2025-02-27T20:10:35.357907Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='67f8ad5c-fdc0-4fad-9203-a9c1c6beb7fe'
    )
]

We can also use the `graph.search` method to search edges/facts for arbitrary text. This API offers more options, including the ability to search node summaries and various re-rankers.

In [8]:
r = zep.graph.search(user_id=user_id, query="Why are there so many goats?", limit=4, scope="edges")
rich.print(r.edges)

[
    EntityEdge(
        created_at='2025-02-27T20:12:46.930245Z',
        episodes=['33651941-0da5-463d-a818-08e0d23187db'],
        expired_at=None,
        fact='The magic pen tool replaces the drawn boat shape with goats.',
        invalid_at=None,
        name='REPLACES_WITH',
        source_node_uuid='117f4280-51e4-4697-b465-0964ddfa5d06',
        target_node_uuid='5b6aa1b1-e58f-423e-9fa5-5ba931457ed3',
        uuid_='b4e9fb04-257a-4f8d-a1f1-4934b7442023',
        valid_at='2025-02-27T20:10:36.568814Z',
        graph_id='fbc4c848-90d1-4e87-9a75-a246559f83aa'
    ),
    EntityEdge(
        created_at='2025-02-27T20:11:09.321202Z',
        episodes=['88538401-18d5-47f0-86e6-5a9ffc0ad57d'],
        expired_at=None,
        fact='The stupid app keeps turning mountains into fountains.',
        invalid_at=None,
        name='TURNS_INTO',
        source_node_uuid='1bff743c-6fb0-45e3-8526-5ee17c854211',
        target_node_uuid='337ec210-a4af-4e0c-b613-aad5588bcda6',
        uuid_='3c1ae74a-97e7-49b0-bef3-51f66c5034a4',
        valid_at='2025-02-27T20:10:35.357907Z',
        graph_id='fbc4c848-90d1-4e87-9a75-a246559f83aa'
    ),
    EntityEdge(
        created_at='2025-02-27T20:11:09.321207Z',
        episodes=['88538401-18d5-47f0-86e6-5a9ffc0ad57d'],
        expired_at=None,
        fact='Emily1130 draws people.',
        invalid_at=None,
        name='DRAWS',
        source_node_uuid='fbc4c848-90d1-4e87-9a75-a246559f83aa',
        target_node_uuid='14e9cc84-f633-47b0-985e-14d343ea8e82',
        uuid_='a0456c3f-1246-4744-a6e4-bc8a409f91e6',
        valid_at='2025-02-27T20:10:35.357907Z',
        graph_id='fbc4c848-90d1-4e87-9a75-a246559f83aa'
    ),
    EntityEdge(
        created_at='2025-02-27T20:11:09.321213Z',
        episodes=['88538401-18d5-47f0-86e6-5a9ffc0ad57d'],
        expired_at=None,
        fact="All the people in Emily1130's drawings have six fingers.",
        invalid_at=None,
        name='HAS',
        source_node_uuid='14e9cc84-f633-47b0-985e-14d343ea8e82',
        target_node_uuid='21109c4f-3f88-4074-b181-cd0b24769cd3',
        uuid_='3d694bbf-10cf-4320-9c0a-1331b8306177',
        valid_at='2025-02-27T20:10:35.357907Z',
        graph_id='fbc4c848-90d1-4e87-9a75-a246559f83aa'
    )
]

## Creating a simple Chatbot

In the next cells, Emily starts a new chat session with a support agent and complains that she can't log in. Our simple chatbot will, given relevant facts retrieved from Zep's graph, respond accordingly.

Here, the support agent is provided with Emily's billing information and account status, which Zep retrieves as most relevant to Emily's login issue.

In [9]:
new_session_id = str(uuid.uuid4())

emily_message = "Hi, I can't log in!"

# We start a new session indicating that Emily has started a new chat with the support agent.
zep.memory.add_session(user_id=user_id, session_id=new_session_id)

# We need to add the Emily's message to the session in order for memory.get to return
# relevant facts related to the message
zep.memory.add(
    session_id=new_session_id,
    messages=[Message(role_type="user", role=user_name, content=emily_message)],
)

AddMemoryResponse(context=None)

In [10]:
system_message = """
You are a customer support agent. Carefully review the facts about the user below and respond to the user's question.
Be helpful and friendly.
"""

memory = zep.memory.get(session_id=new_session_id)

messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "assistant",
        # The context field is an opinionated string that contains facts and entities relevant to the current conversation.
        "content": memory.context,
    },
    {
        "role": "user",
        "content": emily_message,
    },
]

response = oai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
)

print(response.choices[0].message.content)

Hi Emily! I'm sorry to hear that you're having trouble logging in. It looks like your account is currently suspended due to a payment failure, specifically because your card has expired. 

To resolve this, you'll need to update your payment information. If you need help with that or have any other questions, feel free to ask!


Let's look at the memory context string Zep retrieved for the above `memory.get` call.

In [11]:
rich.print(memory.context)

FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - Account with ID 'Emily1130' has a status of 'suspended'. (2025-02-27 20:10:36 - present)
  - user has the id of Emily1130 (2025-02-27 20:10:35 - present)
  - The bug report has been escalated to the engineering team. (2024-03-16 14:27:00 - present)
  - Emily is experiencing issues that she is discussing with SupportBot. (2025-02-27 20:10:35 - present)
  - The account Emily1130 has a failure reason of Card expired. (2024-09-15 00:00:00 - present)
  - User with ID 'Emily1130' has an account with ID 'Emily1130'. (2025-02-27 20:10:36 - present)
  - Emily is a customer seeking assistance from PaintWiz support. (2025-02-27 20:10:35 - present)
  - The support agent responded to Emily's report of the bug. (2024-03-16 14:22:00 - present)
  - The user Emily reported a bug regarding the magic pen tool. (2024-03-16 14:20:00 - present)
  - SupportBot requests more details from Emily about the issues she is facing. (2025-02-27 20:10:35 - present)
  - user has the name of Emily Painter (2025-02-27 20:10:35 - present)
  - SupportBot is assisting Emily in her inquiry at PaintWiz support. (2025-02-27 20:10:35 - present)
  - Emily1130 refers to the app as 'stupid'. (2025-02-27 20:10:35 - present)
  - Emily1130 is using the character creator. (2025-02-27 20:10:35 - present)
  - The account Emily1130 has a card ending in 1234. (2025-02-27 20:10:36 - present)
  - The reason for the account status 'suspended' is 'payment failure'. (2025-02-27 20:10:36 - present)
  - The account Emily1130 has a transaction status of Failed. (2024-08-30 00:00:00 - 2024-09-15 00:00:00)
  - user has the email of Emily@painters.com (2025-02-27 20:10:35 - present)
  - Emily1130 is a user expressing frustration about the AI art generation. (2025-02-27 20:10:35 - present)
  - The stupid app keeps turning mountains into fountains. (2025-02-27 20:10:35 - present)
</FACTS>

# These are the most relevant entities
# ENTITY_NAME: entity summary
<ENTITIES>
  - stupid app: Emily1130 expressed her frustration with the 'stupid app' for AI art generation, stating that it 
consistently misinterprets her requests. Specifically, when she tries to draw mountains, the app turns them into 
fountains, and it inaccurately depicts people in her drawings with six fingers.
  - six fingers: Emily1130 expressed frustration with the AI art generation in the PaintWiz app, specifically 
noting that the app consistently renders people in her drawings with six fingers.
  - failure_reason: Two transactions associated with the account ID 'Emily1130' were marked as 'Failed' due to the 
expiration of the card used for payment. The first transaction occurred on August 30, 2024, for an amount of 
$99.99, and the second transaction took place on September 15, 2024, also for $99.99. In both cases, the last four 
digits of the card were '1234'.
  - landscape generator: Emily1130 is experiencing significant issues with the landscape generator in the app. She 
reports that when attempting to draw mountains, the app incorrectly transforms them into fountains. Additionally, 
she expresses frustration with the character creator, noting that the people in her drawings are depicted with six 
fingers. Emily is very upset and feels that the AI art generation features are completely broken.
  - user_id: Emily reported a bug with the magic pen tool in a drawing application, where attempting to draw boats 
resulted in goats being drawn instead. A support agent acknowledged the issue and requested more details. Emily 
explained her process, and the agent confirmed the bug, stating it would be escalated to the engineering team. As a
temporary workaround, the agent suggested manually selecting the boat shape. Emily expressed hope for a quick 
resolution. Additionally, it was noted that Emily's account (User ID "Emily1130") is currently su